In [7]:
from langchain.document_loaders import *

loader = UnstructuredEPubLoader("japanese.epub")
docs = loader.load()

In [8]:
docs

[Document(page_content='目次\n\n序章\n\n一章：魔王\n\n二章：悪徳の栄え\n\n三章：吾、覇王たらんと欲する者故に\n\n四章：雪花に舞う\n\nエピローグ\n\n「ブランドン、本当に探索者シーカーを辞めるつもりなのか？」\n\n夕焼けに染まる街、二人の男の影法師が、色濃く、そして細く伸びている。燕えん尾び服ふくを着た紳士――ハロルドの問い掛けに、巨大な戦せん斧ぷを背負った甲かつ冑ちゆう姿の大男――ブランドンが振り返った。\n\n「ああ、そのつもりだ。……おまえには、これまで世話になったな」\n\nブランドンのらしくない言葉に、ハロルドは眉を顰ひそめる。\n\n「冥獄十王ヴアリアントの一柱を葬った稀き代たいの大英雄の言葉とは思えないな」\n\n「俺一人の力じゃない。共に戦った全員の成果だ」\n\n「……おまえ、一体どうしたんだ？\u3000探索者シーカーの聖地であるこの帝都で、最強にして最凶と恐れられた不滅の悪鬼オーバーデスが、女一人のために全てを投げ出すだって？\u3000正気か？」\n\nハロルドの問いに、ブランドンは答えない。重い沈黙だけが張り詰めている。\n\nブランドン・シュトーレン、またの名を不滅の悪鬼オーバーデス。帝都最強のクラン、七星レガリアの一等星、血刃連盟に所属するこの大男は、間違いなく破格の大英雄だ。人類史を紐ひも解といても、最上位に君臨するだろうことを、ハロルドは確信している。\n\nだからこそ、冥獄十王ヴアリアントの一柱を葬るという前人未踏の奇跡を起こすこともできた。ブランドンは自分一人の力ではないと言ったが、ブランドンがいなければ確実に敗北していた。ハロルドもまた戦場に立っていたので断言できる。この男は帝国だけでなく、人類そのものを救ったのだ。\n\n初めて会った時は、気に入らない男だと思った。粗野で粗暴、狡こう猾かつで残忍、女にだらしなく、一日中酒浸りのろくでなしだ。とうてい尊敬できる類の男ではない。\n\nだが、ブランドンは強かった。どんな過酷な戦いでも、必ず勝利を収めた。その強さに、ハロルドは心から震えた。恐怖したからではない。ブランドンの圧倒的な強さに感動したからだ。こんなにも強い男がいるのか、と胸が熱くなった。\n\nハロルドは探索者協会シーカーギルドの監察官だ。腕っ節の強さを認められ、十五とい

In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [17]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(docs)):
    texts = text_splitter.split_text(record.page_content)
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'tokens': tiktoken_len(texts[i])
    } for i in range(len(texts))])

100%|██████████| 1/1 [00:00<00:00,  1.76it/s]


In [23]:
totaltoken = 0

for item in chunks :
    totaltoken += int(item['tokens'])

In [24]:
totaltoken
totaltext = ""


276507

In [59]:
import openai
import os

# initialize openai API key
openai.api_key = os.getenv("OPENAI_API_KEY_FREE")

query = "請幫我翻譯成繁體中文，並用繁體中文輸出"


#contexts = [item['metadata']['text'] for item in chunks]
batch_size = 10

# for i in tqdm(range(1100, len(chunks), batch_size)):
#     i_end = min(len(chunks), i+batch_size)
#     meta_batch = chunks[i:i_end]
#     contexts = [x['text'] for x in meta_batch]
#     augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

#     primer = f"""You are translate bot. A highly intelligent system that translate the information provided by the user above"."""

#     res = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": primer},
#             {"role": "user", "content": augmented_query}
#         ]
#     )
#     totaltext = totaltext + "\n" + res['choices'][0]['message']['content']
    



100%|██████████| 4/4 [06:32<00:00, 98.17s/it] 


In [56]:
totaltext

'\n目錄\n\n序章\n\n第一章：魔王\n\n第二章：不道德的繁榮\n\n第三章：為了成為霸王\n\n第四章：舞翩翩的雪花\n\n結語\n\n「布蘭登，你真的打算放棄成為探險者尋求者了嗎？」\n\n在夕陽染紅的城鎮上，兩個男人的影子隨著漸濃漸細的光芒伸長。穿著燕尾服的紳士 - 哈洛德 - 問道，而背負著巨大戰斧，穿著鎧甲的巨漢 - 布蘭登 - 則轉過身來。\n\n---\n\n「是啊，我就是要這樣做... 謝謝你這段時間以來的照顧。」\n\n布蘭登不像往常那樣說話，讓哈洛德皺起了眉頭。\n\n「冥獄十王之一的瓦利安托已經被埋葬，這是一位傑出的大英雄的話嗎？」\n\n「這不是我一個人的力量。是和所有人一起戰鬥的結果。」\n\n「......你到底怎麼了？在探險者尋求者的聖地帝都，那個無敵又兇惡、被人恐懼的殘暴不死魔鬼 - 布魯登·奧巴迪斯，為了一個女人而放棄一切？你是不是瘋了？」\n\n---\n\n對於哈洛德的問題，布魯登沒有回答。厚重的沉默籠罩著兩人。\n\n布魯登·斯托倫，又被稱為不死魔鬼奧巴迪斯。他是帝都最強的七星Regalia中的一等星，屬於血刃聯盟的這位大漢無疑是舉世無雙的大英雄。即使將世界史上的人物作比較，他也肯定居於最頂端。哈洛德深信不疑。\n\n因此，他才能創造前所未有的奇蹟，將冥獄十王之一的瓦利安托擊退。布蘭登說他這樣做不是靠一己之力，但如果布蘭登不在那裡，他們肯定會戰敗。作為戰場上的一員，哈洛德也絕對可以斷言。這個男人救了帝國，也救了人類。\n\n當他們第一次見面時，哈洛德不喜歡這個男人。他粗魯，粗暴，狡詐殘忍，加上在女人面前毫無節制，而且整天沉醉於酒中。他不是值得尊敬的人。\n\n但是，布魯登很強大。他無論面對多麼嚴峻的戰鬥，都能獲得勝利。哈洛德被他的強大震撼到了，不是因為恐懼，而是因為他感動了。他的心受到了這種強大的情感的震動，在胸中熾熱起來。\n\n哈洛德是一名探險者協會的查找者公會監察官。由於他的強大力量，他在15歲就被任命為監察官，從此經歷了許多克蘭和探險者尋求者。他們不僅是強大的人物，而且還有誠懇的心和博愛的精神。在和他們打交道的過程中，哈洛德的人格得到了磨練。\n\n這種堅定的力量就是哈洛德被布魯登所吸引的原因。\n\n當冥獄十王之一的瓦利安托出現時，他先滅了帝國周邊的三個國家，世界上每個人都認為人類即將滅亡。但哈洛德心中不這麼

In [60]:
def writefile(fname, text):
    try:
        with open(fname, 'a+', encoding='utf-8') as f:
            return f.write(text)
    except:
        return None

writefile("翻譯本2.md", totaltext)

146276

In [43]:


augmented_query

'目次\n\n序章\n\n一章：魔王\n\n二章：悪徳の栄え\n\n三章：吾、覇王たらんと欲する者故に\n\n四章：雪花に舞う\n\nエピローグ\n\n「ブランドン、本当に探索者シーカーを辞めるつもりなのか？」\n\n夕焼けに染まる街、二人の男の影法師が、色濃く、そして細く伸びている。燕えん尾び服ふくを着た紳士――ハロルドの問い掛けに、巨大な戦せん斧ぷを背負った甲かつ冑ちゆう姿の大男――ブランドンが振り返った。\n\n---\n\n「ああ、そのつもりだ。……おまえには、これまで世話になったな」\n\nブランドンのらしくない言葉に、ハロルドは眉を顰ひそめる。\n\n「冥獄十王ヴアリアントの一柱を葬った稀き代たいの大英雄の言葉とは思えないな」\n\n「俺一人の力じゃない。共に戦った全員の成果だ」\n\n「……おまえ、一体どうしたんだ？\u3000探索者シーカーの聖地であるこの帝都で、最強にして最凶と恐れられた不滅の悪鬼オーバーデスが、女一人のために全てを投げ出すだって？\u3000正気か？」\n\n---\n\nハロルドの問いに、ブランドンは答えない。重い沈黙だけが張り詰めている。\n\nブランドン・シュトーレン、またの名を不滅の悪鬼オーバーデス。帝都最強のクラン、七星レガリアの一等星、血刃連盟に所属するこの大男は、間違いなく破格の大英雄だ。人類史を紐ひも解といても、最上位に君臨するだろうことを、ハロルドは確信している。\n\n---\n\nだからこそ、冥獄十王ヴアリアントの一柱を葬るという前人未踏の奇跡を起こすこともできた。ブランドンは自分一人の力ではないと言ったが、ブランドンがいなければ確実に敗北していた。ハロルドもまた戦場に立っていたので断言できる。この男は帝国だけでなく、人類そのものを救ったのだ。\n\n初めて会った時は、気に入らない男だと思った。粗野で粗暴、狡こう猾かつで残忍、女にだらしなく、一日中酒浸りのろくでなしだ。とうてい尊敬できる類の男ではない。\n\n---\n\nだが、ブランドンは強かった。どんな過酷な戦いでも、必ず勝利を収めた。その強さに、ハロルドは心から震えた。恐怖したからではない。ブランドンの圧倒的な強さに感動したからだ。こんなにも強い男がいるのか、と胸が熱くなった。\n\n---\n\nハロルドは探索者協会シーカーギルドの監察官だ。腕っ節の

In [50]:
# system message to 'prime' the model


from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

「不要走，布蘭登。帝國需要你。你雖然戰勝冥獄十王瓦里安特，但人民的內心仍然充滿恐懼與不安。他們需要一個能給予勇氣的象徵。而唯一能夠勝任這個角色的，就只有你。」

哈洛德認真地說道。但布蘭登搖了搖頭。

「......我已經做出決定了。我要辭去探險家的職務。」

瞬間，哈洛德因憤怒而無法思考。

!! 你到底有多自私啊！

--

!? 我知道你很擔心你喜歡的女孩，但現在不是時候吧！ 大家都需要你！

!? 然而，現在你卻要後腳刨着沙子離去嗎？

!? 你真的認為這樣可以被允許嗎？

!? 回答我，布蘭登·斯托倫！ 不，讓我來吼：

!!

他因情感而大聲喊叫。他無法不喊叫。

「......對不起。」

--

然而，哈洛德的想法並沒有被傳達給布蘭登。面對哈洛德生氣的樣子，布蘭登的頭低了下來。在他受傷的臉上，哈洛德用全力揮拳。布蘭登的口中流血了。哈洛德準備好反擊，但布蘭登卻靜靜地站在那裡。在被打的憤怒之外，他臉上只剩下一個孤獨的微笑。

--

「你是認真的啊......」

哈洛德理解了布蘭登的堅定決心。布蘭登走近他，從口袋中拿出了一個紅色的吊墜。吊墜有二把劍和一把斧交錯疊合的形狀。這是血刃聯盟的紋章，也是證明他們成員身份的吊飾。

--

「這個，我送給你。我曾經嘗試還給主人，但他不要了。但現在，我沒有這個資格了。所以，我把它託付給你。」

「......我也不需要。」

當哈洛德失敗地試圖反抗時，布蘭登帶着困惑笑了。 

「不要那樣說。唯一能夠達成這個要求的人就是你。」

硬生生的讓哈洛德握住了吊墜。他試圖抗議，但布蘭登轉身逃跑，讓哈洛德感到非常憤怒。

--

「再見，哈洛德。好好珍惜你的妻子和孩子。」

布蘭登通過肩膀向哈洛德揮手。對於他自私無情的行為，哈洛德把手中的吊墜扔了過去。 然而，他做不到。

「......你這個笨蛋。」

從握緊的手掌中流出了血。從那一天開始，哈洛德再也沒有見過布蘭登。

數十年的時光流逝了。

--

年老的哈洛德在常去的咖啡館里品味紅茶，陷入了回憶。他的手上拿着一個朋友託付給他的吊墜。這個白銀米斯特製造的吊墜，在幾十年的時間里，一直保持着燦爛的光芒。

然而，血刃聯盟早已解散，持有這個吊飾的布蘭登也在和魔王路德的戰斗中死去了。時間的流逝是殘酷的。它會奪走我們所有珍貴的東西。殘留下來的只有逐漸逝去的生命。

--

直到最近，他一直陷入悲傷之中。但現在不同了。

「如果我能活得更久的話，也許會發生好事。」

哈洛德嘟囔一句，露出了輕松的微笑。他的腦海中浮現着一個年輕人的身影。黑色長袍的青年被稱為「蛇」，臉上露出嘲諷的笑容。

「你懂我的意思嗎？」

「你好像很高興呢？」

突然間，有個女孩從旁邊喊了他一聲。他是這家咖啡館的常客，和這個女孩也很熟。不過，他們之間只有閒話家常的關係。

「你知道嗎？」

「我知道啊～。哈洛德先生，就在不久之前還總是顯得非常孤獨，現在好像變得非常有活力了。難道是找到了戀人？」

「差一點！我是永遠忠於亡妻的愛情呢。」

「嗯？」女孩疑惑地問道。「哈洛德先生這麼帥，難道不會有很多女孩子喜歡嗎？」 

「嘿嘿，只有你會這麼說，小姐。」


「不要走，布蘭登。帝國需要你。你雖然戰勝冥獄十王瓦里安特，但人民的內心仍然充滿恐懼與不安。他們需要一個能給予勇氣的象徵。而唯一能夠勝任這個角色的，就只有你。」

哈洛德認真地說道。但布蘭登搖了搖頭。

「......我已經做出決定了。我要辭去探險家的職務。」

瞬間，哈洛德因憤怒而無法思考。

!! 你到底有多自私啊！

--

!? 我知道你很擔心你喜歡的女孩，但現在不是時候吧！ 大家都需要你！

!? 然而，現在你卻要後腳刨着沙子離去嗎？

!? 你真的認為這樣可以被允許嗎？

!? 回答我，布蘭登·斯托倫！ 不，讓我來吼：

!!

他因情感而大聲喊叫。他無法不喊叫。

「......對不起。」

--

然而，哈洛德的想法並沒有被傳達給布蘭登。面對哈洛德生氣的樣子，布蘭登的頭低了下來。在他受傷的臉上，哈洛德用全力揮拳。布蘭登的口中流血了。哈洛德準備好反擊，但布蘭登卻靜靜地站在那裡。在被打的憤怒之外，他臉上只剩下一個孤獨的微笑。

--

「你是認真的啊......」

哈洛德理解了布蘭登的堅定決心。布蘭登走近他，從口袋中拿出了一個紅色的吊墜。吊墜有二把劍和一把斧交錯疊合的形狀。這是血刃聯盟的紋章，也是證明他們成員身份的吊飾。

--

「這個，我送給你。我曾經嘗試還給主人，但他不要了。但現在，我沒有這個資格了。所以，我把它託付給你。」

「......我也不需要。」

當哈洛德失敗地試圖反抗時，布蘭登帶着困惑笑了。 

「不要那樣說。唯一能夠達成這個要求的人就是你。」

硬生生的讓哈洛德握住了吊墜。他試圖抗議，但布蘭登轉身逃跑，讓哈洛德感到非常憤怒。

--

「再見，哈洛德。好好珍惜你的妻子和孩子。」

布蘭登通過肩膀向哈洛德揮手。對於他自私無情的行為，哈洛德把手中的吊墜扔了過去。 然而，他做不到。

「......你這個笨蛋。」

從握緊的手掌中流出了血。從那一天開始，哈洛德再也沒有見過布蘭登。

數十年的時光流逝了。

--

年老的哈洛德在常去的咖啡館里品味紅茶，陷入了回憶。他的手上拿着一個朋友託付給他的吊墜。這個白銀米斯特製造的吊墜，在幾十年的時間里，一直保持着燦爛的光芒。

然而，血刃聯盟早已解散，持有這個吊飾的布蘭登也在和魔王路德的戰斗中死去了。時間的流逝是殘酷的。它會奪走我們所有珍貴的東西。殘留下來的只有逐漸逝去的生命。

--

直到最近，他一直陷入悲傷之中。但現在不同了。

「如果我能活得更久的話，也許會發生好事。」

哈洛德嘟囔一句，露出了輕松的微笑。他的腦海中浮現着一個年輕人的身影。黑色長袍的青年被稱為「蛇」，臉上露出嘲諷的笑容。

「你懂我的意思嗎？」

「你好像很高興呢？」

突然間，有個女孩從旁邊喊了他一聲。他是這家咖啡館的常客，和這個女孩也很熟。不過，他們之間只有閒話家常的關係。

「你知道嗎？」

「我知道啊～。哈洛德先生，就在不久之前還總是顯得非常孤獨，現在好像變得非常有活力了。難道是找到了戀人？」

「差一點！我是永遠忠於亡妻的愛情呢。」

「嗯？」女孩疑惑地問道。「哈洛德先生這麼帥，難道不會有很多女孩子喜歡嗎？」 

「嘿嘿，只有你會這麼說，小姐。」